<a href="https://colab.research.google.com/github/vishxl/Book-Price-Prediction/blob/master/Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import requests, zipfile, io
r = requests.get( 'https://github.com/vishxl/Book-Price-Prediction/blob/master/Data.zip?raw=true' ) 
z = zipfile.ZipFile(io.BytesIO(r.content)) 
z.extractall()

In [29]:
import pandas as pd
import numpy as np

In [30]:
train = pd.read_excel('Participants_Data/Data_Train.xlsx')
test = pd.read_excel('Participants_Data/Data_Test.xlsx')

sub = pd.read_excel('Participants_Data/Sample_Submission.xlsx')

In [31]:
train.head().T

,0,1,2,3,4
Title,The Prisoner's Gold (The Hunters 3),Guru Dutt: A Tragedy in Three Acts,Leviathan (Penguin Classics),A Pocket Full of Rye (Miss Marple),LIFE 70 Years of Extraordinary Photography
Author,Chris Kuzneski,Arun Khopkar,Thomas Hobbes,Agatha Christie,Editors of Life
Edition,"Paperback,– 10 Mar 2016","Paperback,– 7 Nov 2012","Paperback,– 25 Feb 1982","Paperback,– 5 Oct 2017","Hardcover,– 10 Oct 2006"
Reviews,4.0 out of 5 stars,3.9 out of 5 stars,4.8 out of 5 stars,4.1 out of 5 stars,5.0 out of 5 stars
Ratings,8 customer reviews,14 customer reviews,6 customer reviews,13 customer reviews,1 customer review
Synopsis,THE HUNTERS return in their third brilliant no...,A layered portrait of a troubled genius for wh...,"""During the time men live without a common Pow...",A handful of grain is found in the pocket of a...,"For seven decades, ""Life"" has been thrilling t..."
Genre,Action & Adventure (Books),Cinema & Broadcast (Books),International Relations,Contemporary Fiction (Books),Photography Textbooks
BookCategory,Action & Adventure,"Biographies, Diaries & True Accounts",Humour,"Crime, Thriller & Mystery","Arts, Film & Photography"
Price,220,202.93,299,180,965.62


In [32]:
print(train.Title)

0                     The Prisoner's Gold (The Hunters 3)
1                      Guru Dutt: A Tragedy in Three Acts
2                            Leviathan (Penguin Classics)
3                      A Pocket Full of Rye (Miss Marple)
4              LIFE 70 Years of Extraordinary Photography
                              ...                        
6232    Humans: A Brief History of How We F*cked It Al...
6233                                          The Chemist
6234    The Duke And I: Number 1 in series (Bridgerton...
6235                         Frostfire (Kanin Chronicles)
6236                          The First Order (Sam Capra)
Name: Title, Length: 6237, dtype: object


In [33]:
#combining test and train for cleaning

combined = pd.concat([train, test], sort=False)
combined.reset_index(drop=True, inplace=True)

In [34]:
combined.index

RangeIndex(start=0, stop=7797, step=1)

In [35]:
combined.values

array([["The Prisoner's Gold (The Hunters 3)", 'Chris Kuzneski',
        'Paperback,– 10 Mar 2016', ..., 'Action & Adventure (Books)',
        'Action & Adventure', 220.0],
       ['Guru Dutt: A Tragedy in Three Acts', 'Arun Khopkar',
        'Paperback,– 7 Nov 2012', ..., 'Cinema & Broadcast (Books)',
        'Biographies, Diaries & True Accounts', 202.93],
       ['Leviathan (Penguin Classics)', 'Thomas Hobbes',
        'Paperback,– 25 Feb 1982', ..., 'International Relations',
        'Humour', 299.0],
       ...,
       ['The Kite Runner Graphic Novel', 'Khaled Hosseini',
        'Paperback,– 6 Sep 2011', ..., 'Humour (Books)', 'Humour', nan],
       ['Panzer Leader (Penguin World War II Collection)',
        'Heinz Guderian', 'Paperback,– 22 Sep 2009', ...,
        'United States History', 'Biographies, Diaries & True Accounts',
        nan],
       ['Complete Spanish Step-by-Step', 'Barbara Bregstein',
        'Paperback,– 16 Sep 2016', ..., 'Dictionaries',
        'Language, Lin

In [36]:
combined.shape

(7797, 9)

In [37]:
combined.info() #confirms no missing data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7797 entries, 0 to 7796
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         7797 non-null   object 
 1   Author        7797 non-null   object 
 2   Edition       7797 non-null   object 
 3   Reviews       7797 non-null   object 
 4   Ratings       7797 non-null   object 
 5   Synopsis      7797 non-null   object 
 6   Genre         7797 non-null   object 
 7   BookCategory  7797 non-null   object 
 8   Price         6237 non-null   float64
dtypes: float64(1), object(8)
memory usage: 548.4+ KB


In [38]:
combined['Reviews_values'] = combined['Reviews'].str.split(' ').str.get(0).astype(float)
combined['Rating_values'] = combined['Ratings'].str.split(' ').str.get(0).str.replace(',','').astype(float)

In [39]:
combined[['Issue_type','Issue_date']] = combined['Edition'].str.split(',– ',expand=True)

combined['Issue_type'] = combined.Issue_type.str.extract(r'(^[a-zA-Z|\s]*)')[0]
#.unique()
#.value_counts()
combined['Issue_type'].unique()

array(['Paperback', 'Hardcover', 'Mass Market Paperback', 'Sheet music',
       'Flexibound', 'Plastic Comb', 'Loose Leaf', 'Tankobon Softcover',
       'Perfect Paperback', 'Board book', 'Cards', 'Spiral', '',
       'Product Bundle', 'Library Binding', 'Leather Bound'], dtype=object)

In [40]:
#Month & Year
combined['Issue_year'] = combined.Issue_date.str[-4:]

combined['Issue_month'] = pd.to_datetime(combined['Issue_date'], errors='coerce').dt.month

combined['Issue_month'].value_counts()

1.0     976
10.0    706
9.0     605
5.0     586
11.0    562
4.0     526
6.0     518
3.0     502
7.0     489
8.0     475
12.0    435
2.0     428
Name: Issue_month, dtype: int64

In [41]:
combined['Genre'] = combined['Genre'].str.replace(r"\(.*\)","")  #removing '(Books)' from Genre 

In [42]:
combined.describe(include = 'all').head(2)

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,Reviews_values,Rating_values,Issue_type,Issue_date,Issue_year,Issue_month
count,7797,7797,7797,7797,7797,7797,7797,7797,6237.0,7797.0,7797.0,7797,7797,7797,6808.0
unique,6787,4372,3882,36,368,6771,363,11,NaN,NaN,NaN,16,3332,67,NaN


In [43]:
combined['Rating_Ratio'] = round(combined['Rating_values']/combined['Reviews_values'], 2)

In [44]:
combined['Issue_type'].unique()

array(['Paperback', 'Hardcover', 'Mass Market Paperback', 'Sheet music',
       'Flexibound', 'Plastic Comb', 'Loose Leaf', 'Tankobon Softcover',
       'Perfect Paperback', 'Board book', 'Cards', 'Spiral', '',
       'Product Bundle', 'Library Binding', 'Leather Bound'], dtype=object)

In [61]:
#Authors
author_replacements = {' & ':', ',"0":"other","2":"other",'A. P. J. Abdul Kalam':'A.P.J. Abdul Kalam','APJ Abdul Kalam':'A.P.J. Abdul Kalam','Agrawal P. K.': 'Agrawal P.K','Ajay K Pandey': 'Ajay K. Pandey','Aravinda Anantharaman': 'Aravinda Anatharaman','Arthur Conan Doyle': 'Sir Arthur Conan Doyle','B A Paris': 'B. A. Paris','E L James': 'E. L. James','E.L. James':'E. L. James','Eliyahu M Goldratt': 'Eliyahu M. Goldratt','Ernest Hemingway': 'Ernest Hemmingway','Frank Miler': 'Frank Miller','Fyodor Dostoevsky': 'Fyodor Dostoyevsky','George R R Martin': 'George R. R. Martin','George R.R. Martin':'George R. R. Martin','H. G. Wells': 'H.G. Wells','Johann Wolfgang Von Goethe': 'Johann Wolfgang von Goethe','John Le Carré': 'John le Carré','Judith McNaught': 'Judith Mcnaught','Keith Giffen': 'Kieth Giffen','Ken Hultgen': 'Ken Hultgren','Kentaro Miura': 'Kenturo Miura','Kohei Horikoshi': 'Kouhei Horikoshi','M.K Gandhi': 'M.K. Gandhi','Matthew K Manning': 'Matthew Manning','Michael Crichton': 'Micheal Crichton','N.K Aggarwala': 'N.K. Aggarwala','Oxford University Press (India)': 'Oxford University Press India','P D James': 'P. D. James','Paramahansa Yogananda': 'Paramhansa Yogananda','R K Laxman': 'R. K. Laxman','R.K. Laxman': 'R. K. Laxman','R. M. Lala': 'R.M. Lala','Raina Telgemaeier': 'Raina Telgemeier','Rajaraman': 'Rajaraman V','Rajiv M. Vijayakar': 'Rajiv Vijayakar','Ramachandra Guha': 'Ramchandra Guha','Rene Goscinny': 'René Goscinny','Richard P Feynman': 'Richard P. Feynman','S Giridhar': 'S. Giridhar','S Hussain Zaidi': 'S. Hussain Zaidi','S. A. Chakraborty': 'S. Chakraborty','Santosh Kumar K': 'Santosh Kumar K.',"S.C. Gupta" : "S. C. Gupta",'Shiv Prasad Koirala': 'Shivprasad Koirala','Shivaprasad Koirala': 'Shivprasad Koirala','Simone De Beauvoir': 'Simone de Beauvoir','Sir Arthur Conan Doyle': 'Arthur Conan Doyle',"Terry O' Brien": "Terry O'Brien",'Thich Nhat Hahn': 'Thich Nhat Hanh','Trinity College Lond': 'Trinity College London',"Trinity College London Press" : "Trinity College London",'Ursula K. Le Guin': 'Ursula Le Guin','Willard A Palmer': 'Willard A. Palmer','Willard Palmer': 'Willard A. Palmer','William Strunk Jr': 'William Strunk Jr.','Yashavant Kanetakr': 'Yashavant Kanetkar','Yashavant P. Kanetkar': 'Yashavant Kanetkar','Yashwant Kanetkar': 'Yashavant Kanetkar','et al': 'et al.',' et al': 'et al.','Peter Clutterbuck': ' Peter Clutterbuck','Scholastic': 'Scholastic ','Ullekh N. P.': 'Ullekh N.P.','Shalini Jain': 'Dr. Shalini Jain','Kevin Mitnick': 'Kevin D. Mitnick'}
combined['Author'] = combined['Author'].replace(author_replacements,regex=True)
combined['Author_num'] = combined['Author'].apply(lambda x: len(x.split(',')))

In [65]:
#Dropping Columns

#combined.drop(['No_Authors'], axis=1, inplace=True)

In [66]:
combined.head(5)

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,Reviews_values,Rating_values,Issue_type,Issue_date,Issue_year,Issue_month,Rating_Ratio,Author_num
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0 out of 5 stars,8 customer reviews,THE HUNTERS return in their third brilliant no...,Action & Adventure,Action & Adventure,220.00,4.0,8.0,Paperback,10 Mar 2016,2016,3.0,2.00,1
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9 out of 5 stars,14 customer reviews,A layered portrait of a troubled genius for wh...,Cinema & Broadcast,"Biographies, Diaries & True Accounts",202.93,3.9,14.0,Paperback,7 Nov 2012,2012,11.0,3.59,1
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8 out of 5 stars,6 customer reviews,"""During the time men live without a common Pow...",International Relations,Humour,299.00,4.8,6.0,Paperback,25 Feb 1982,1982,2.0,1.25,1
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1 out of 5 stars,13 customer reviews,A handful of grain is found in the pocket of a...,Contemporary Fiction,"Crime, Thriller & Mystery",180.00,4.1,13.0,Paperback,5 Oct 2017,2017,10.0,3.17,1
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0 out of 5 stars,1 customer review,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62,5.0,1.0,Hardcover,10 Oct 2006,2006,10.0,0.20,1


In [48]:
#Dropping Columns

#df_modified.drop(['Edition','Reviews','Ratings'], axis=1, inplace=True)

In [49]:

#converting into categorical data

#df_modified["Book_Type"] = df_modified["Book_Type"].astype('category')
#df_modified.dtypes


In [50]:

#label encoding

#df_modified['Book_Type_Cat'] = df_modified["Book_Type"].cat.codes


In [51]:
train.isnull().sum()

Title           0
Author          0
Edition         0
Reviews         0
Ratings         0
Synopsis        0
Genre           0
BookCategory    0
Price           0
dtype: int64

In [67]:
combined.columns

Index(['Title', 'Author', 'Edition', 'Reviews', 'Ratings', 'Synopsis', 'Genre',
       'BookCategory', 'Price', 'Reviews_values', 'Rating_values',
       'Issue_type', 'Issue_date', 'Issue_year', 'Issue_month', 'Rating_Ratio',
       'Author_num'],
      dtype='object')

In [103]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
combined[['BookCategory','Issue_type','Issue_month','Issue_type','Issue_year']] = \
combined[['BookCategory','Issue_type','Issue_month','Issue_type','Issue_year']]\
    .apply(enc.fit_transform)

In [104]:
train = df[df['Price'].notna()]
test = df[df['Price'].isna()]
test.drop(['Price'], axis=1, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
train = train[train['Price'] <= 12000]

In [ ]:
X = train.loc[:, train.columns != 'Price'].values
X = X.astype(float)

# Dependent Variable

y = np.log1p(train['Price'].values)
y = y.astype(float)

# Test - (Independent Variables)

test = test.loc[:].values
test = test.astype(float)

Machine Learning


In [ ]:
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import make_scorer
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection \
    import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble \
    import RandomForestRegressor, VotingRegressor, AdaBoostRegressor

In [ ]:
#RF
rf = RandomForestRegressor(random_state=0,bootstrap=False,max_features='sqrt')

cvs = cross_val_score(rf, X, y, cv=5,verbose=2,n_jobs=-1,
                      scoring=make_scorer(score,greater_is_better=True))
mean_score = sum(cvs)/len(cvs)
print("Average Score:",mean_score)